# Read a directory of geoTIFF Files
The process for reading a singe GeoTIFF file can be repeated in a loop to import a time series of GeoTIFF files for the same location. For this to work, the time-step between consecutive files, the coordinates of the area covered, and the spatial resolution should be consistent for the whole time series. `glob` form the *glob* library is used to read the names of the directory of files. The files should be named so that the names will form a sequence that matches the time series e.g. 2020-01-31.tiff, 2020-02-28.tiff ...

## Setup

In [11]:
import xarray as xr
import pandas as pd # For creating time series objects
import rioxarray # For reading GeoTIFFs
import glob # For reading a directory of file names matching a pattern

## Read the Directory of File Names

In [5]:
file_names = glob('Data/GeoTIFF/*.tiff')
file_names

['Data/GeoTIFF\\NDVI_2018.tiff',
 'Data/GeoTIFF\\NDVI_2019.tiff',
 'Data/GeoTIFF\\NDVI_2020.tiff']

## Create the Data Array
A `for` loop is used to read in each GeoTIFF and add it to a list. The list is then concatenated usning the Xarray `concat` method along a new dimension called 'year'. This produces a data array where the year dimension lacks coordinates, plus an unwanted dimension called 'band'. The data array `assign_coords()` method can be used in conjunction with Pandas `date_range()` function to add datetime64 coordinates to the year dimension. The unwanted 'band' dimension can be dropped using the data array indexing using the `.sel()` method.

In [62]:
# Create an empty list to hold the GeoTIFFS
files=[]

# Loop through the file names and append each file to the list
for file in file_names:
    files.append(rioxarray.open_rasterio(file))

# Concatenate the list along the a new 'time' dimension to create the data array
da = xr.concat(files, dim='year')

# Show a summary of the data array
da

<xarray.DataArray (year: 3, band: 1, y: 200, x: 170)>
array([[[[0.61551064, 0.65427697, 0.7156274 , ..., 0.72888285,
          0.7252962 , 0.76970327],
         [0.70380235, 0.65357006, 0.7174596 , ..., 0.76404494,
          0.70277715, 0.78298575],
         [0.6674025 , 0.66975695, 0.7092199 , ..., 0.77210176,
          0.7472504 , 0.7755978 ],
         ...,
         [0.63414633, 0.6853397 , 0.7204512 , ..., 0.7657143 ,
          0.7495737 , 0.76942253],
         [0.66376305, 0.6684574 , 0.7158831 , ..., 0.7358214 ,
          0.76910603, 0.7917882 ],
         [0.71014494, 0.68527186, 0.67096776, ..., 0.75468165,
          0.75141   , 0.7779661 ]]],


       [[[0.6046243 , 0.6670168 , 0.74427134, ..., 0.73944134,
          0.73808444, 0.7712572 ],
         [0.711499  , 0.6663113 , 0.73499537, ..., 0.76585156,
          0.71652097, 0.78657633],
         [0.64146763, 0.663565  , 0.6699739 , ..., 0.7654894 ,
...
          0.6058677 , 0.7222222 ],
         [0.45963508, 0.6450094 , 0.69227165, ..., 0.5288365 ,
          0.67936546, 0.7536644 ],
         [0.674567  , 0.64639217, 0.6061102 , ..., 0.4936306 ,
          0.65233475, 0.67472035]]],


       [[[0.18075341, 0.21125805, 0.26982012, ..., 0.5935089 ,
          0.51826924, 0.598607  ],
         [0.283045  , 0.23946036, 0.24755062, ..., 0.55718434,
          0.4566316 , 0.5360302 ],
         [0.30200946, 0.23763648, 0.2276903 , ..., 0.60900074,
          0.4993046 , 0.57906157],
         ...,
         [0.6644458 , 0.7393478 , 0.7716625 , ..., 0.76928455,
          0.6994404 , 0.760319  ],
         [0.6414583 , 0.707457  , 0.7367367 , ..., 0.71006566,
          0.7544904 , 0.7978678 ],
         [0.7761194 , 0.6947195 , 0.6704712 , ..., 0.7074468 ,
          0.71772724, 0.749499  ]]]], dtype=float32)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 9.604e+06 9.604e+06 9.604e+06 ... 9.62e+06 9.62e+06
  * y            (y) float64 4.466e+06 4.466e+06 ... 4.447e+06 4.447e+06
    spatial_ref  int32 0
Dimensions without coordinates: year
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [63]:
# Create the year datetime64 coordinates
year_coords = pd.date_range('2018', '2021', freq='Y')
year_coords

DatetimeIndex(['2018-12-31', '2019-12-31', '2020-12-31'], dtype='datetime64[ns]', freq='A-DEC')

In [64]:
# Add the datetime64 coordinates to the year dimension

print('Before:\n'+ str(da.year.values)+'\n') # The year dimension before assigning datetime coordinates

# Assign the  coordinates
da = da.assign_coords({'year': year_coords})

print('After:\n'+ str(da.year.values)+'\n')  # The year dimension after assigning coordinates

Before:
[0 1 2]

After:
['2018-12-31T00:00:00.000000000' '2019-12-31T00:00:00.000000000'
 '2020-12-31T00:00:00.000000000']



In [65]:
# Drop the unwanted 'band' dimension by indexing on the single value of band.
da = da.sel(band=1)
da

<xarray.DataArray (year: 3, y: 200, x: 170)>
array([[[0.61551064, 0.65427697, 0.7156274 , ..., 0.72888285,
         0.7252962 , 0.76970327],
        [0.70380235, 0.65357006, 0.7174596 , ..., 0.76404494,
         0.70277715, 0.78298575],
        [0.6674025 , 0.66975695, 0.7092199 , ..., 0.77210176,
         0.7472504 , 0.7755978 ],
        ...,
        [0.63414633, 0.6853397 , 0.7204512 , ..., 0.7657143 ,
         0.7495737 , 0.76942253],
        [0.66376305, 0.6684574 , 0.7158831 , ..., 0.7358214 ,
         0.76910603, 0.7917882 ],
        [0.71014494, 0.68527186, 0.67096776, ..., 0.75468165,
         0.75141   , 0.7779661 ]],

       [[0.6046243 , 0.6670168 , 0.74427134, ..., 0.73944134,
         0.73808444, 0.7712572 ],
        [0.711499  , 0.6663113 , 0.73499537, ..., 0.76585156,
         0.71652097, 0.78657633],
        [0.64146763, 0.663565  , 0.6699739 , ..., 0.7654894 ,
         0.7473684 , 0.77572346],
...
        [0.6528346 , 0.703259  , 0.7078626 , ..., 0.7174832 ,
         0.6058677 , 0.7222222 ],
        [0.45963508, 0.6450094 , 0.69227165, ..., 0.5288365 ,
         0.67936546, 0.7536644 ],
        [0.674567  , 0.64639217, 0.6061102 , ..., 0.4936306 ,
         0.65233475, 0.67472035]],

       [[0.18075341, 0.21125805, 0.26982012, ..., 0.5935089 ,
         0.51826924, 0.598607  ],
        [0.283045  , 0.23946036, 0.24755062, ..., 0.55718434,
         0.4566316 , 0.5360302 ],
        [0.30200946, 0.23763648, 0.2276903 , ..., 0.60900074,
         0.4993046 , 0.57906157],
        ...,
        [0.6644458 , 0.7393478 , 0.7716625 , ..., 0.76928455,
         0.6994404 , 0.760319  ],
        [0.6414583 , 0.707457  , 0.7367367 , ..., 0.71006566,
         0.7544904 , 0.7978678 ],
        [0.7761194 , 0.6947195 , 0.6704712 , ..., 0.7074468 ,
         0.71772724, 0.749499  ]]], dtype=float32)
Coordinates:
    band         int32 1
  * x            (x) float64 9.604e+06 9.604e+06 9.604e+06 ... 9.62e+06 9.62e+06
  * y            (y) float64 4.466e+06 4.466e+06 ... 4.447e+06 4.447e+06
    spatial_ref  int32 0
  * year         (year) datetime64[ns] 2018-12-31 2019-12-31 2020-12-31
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

## Further Reading
See Michael Delgado's answer to this query on Stack Overflow for further information about dropping dimensions: <br> https://stackoverflow.com/questions/61003361/drop-coordinate-from-an-xarray-dataarray/63478532#63478532